In [ ]:
markdown_strs = ['## Consistency']

consistency_file = join(output_dir, '{}_consistency.{}'.format(experiment_id, file_format))
degradation_file = join(output_dir, '{}_degradation.{}'.format(experiment_id, file_format))
disattenuation_file = join(output_dir, '{}_disattenuated_correlations.{}'.format(experiment_id, file_format))
eval_file = join(output_dir, '{}_eval.{}'.format(experiment_id,
                                                 file_format))
confmat_h1h2_file = join(output_dir, '{}_confMatrix_h1h2.{}'.format(experiment_id, file_format))

if exists(consistency_file) and exists(degradation_file) and exists(disattenuation_file):
    df_consistency = DataReader.read_from_file(consistency_file, index_col=0)
    df_degradation = DataReader.read_from_file(degradation_file, index_col=0)
    df_dis_corrs = DataReader.read_from_file(disattenuation_file, index_col=0)
    df_eval = DataReader.read_from_file(eval_file, index_col=0)

    markdown_strs.append('*Note: this section assumes that the score used for evaluating machine scores '
                         'is the score assigned by the first rater.*')
    markdown_strs.append('### Human-human agreement')
    markdown_strs.append("This table shows the human-human agreement on the "
                         "double-scored evaluation data.")
    if continuous_human_score:
        markdown_strs.append('For the computation of `kappa` and `wtkappa` '
                             'human scores have beeen rounded to the nearest integer.')
        
    markdown_strs.append("The following are <span class='highlight_color'>highlighted </span>: ")
    markdown_strs.append(' - Exact agreement (`exact_agr`) < 50%')
    markdown_strs.append(' - Adjacent agreement (`adj_agr`) < 95%')
    markdown_strs.append(' - Quadratic weighted kappa (`wtkappa`) < 0.7')
    markdown_strs.append(' - Pearson correlation (`corr`) < 0.7')
    display(Markdown('\n'.join(markdown_strs)))
    
    # display the HTML for the table with the various formatters
    formatter_exact_agr = partial(color_highlighter, low=50, high=100)
    formatter_adj_agr = partial(color_highlighter, low=95, high=100)
    formatter_wtkappa_corr = partial(color_highlighter, low=0.7)
    formatter_dict = {'exact_agr': formatter_exact_agr, 
                      'adj_agr': formatter_adj_agr,
                      'wtkappa': formatter_wtkappa_corr, 
                      'corr': formatter_wtkappa_corr}
    display(HTML(df_consistency.to_html(index=False,
                                        escape=False,
                                        float_format=float_format_func,
                                        formatters=formatter_dict)))
    
    markdown_strs = ['### Degradation']
    markdown_strs.append('The next table shows the degradation in the evaluation metrics '
                         '(`diff`) when comparing the machine (`H-M`) to a second human (`H-H`). '
                         'A positive degradation value indicates better human-machine performance. '
                         'Note that the human-machine agreement is computed on the full '
                         'dataset (to get a reliable estimate) whereas the human-human '
                         'agreement is computed on the subset of responses that were double-scored.')
    markdown_strs.append("\nThe following degradation values are "
                         "<span class='highlight_color'>highlighted</span>")
    markdown_strs.append(' - `corr` < -0.1')
    markdown_strs.append(' - `wtkappa` < -0.1')
    display(Markdown('\n'.join(markdown_strs)))
    df_eval_for_degradation = df_eval[df_degradation.columns].copy()
    df_consistency_for_degradation = pd.concat([df_consistency]*len(df_eval), sort=True)
    df_consistency_for_degradation = df_consistency_for_degradation[df_degradation.columns].copy()
    df_consistency_for_degradation.index = df_eval_for_degradation.index

    df_consistency_for_degradation['type'] = 'H-H'
    df_eval_for_degradation['type'] = 'H-M'
    df_degradation['type'] = 'diff'

    df = pd.concat([df_consistency_for_degradation,
                    df_eval_for_degradation,
                    df_degradation], sort=True)
    df = df[['type','corr', 'kappa', 'wtkappa', 'exact_agr', 'adj_agr', 'SMD']]
    df = df.reset_index()
    df = df.set_index(['index', 'type']).sort_index(level='index')
    df.index.names = [None, None]
    
    # display the HTML for the table with the various formatters
    formatter_corr = partial(color_highlighter, low=-0.1, high=100)
    formatter_wtkappa = partial(color_highlighter, low=-0.1, high=100)
    formatter_dict = {'corr': formatter_corr, 'wtkappa': formatter_wtkappa}
    display(HTML(df.to_html(float_format=float_format_func, 
                            formatters=formatter_dict, escape=False)))

    if exists(confmat_h1h2_file):
        markdown_strs = ['### Human-human confusion matrix']
        markdown_strs.append("Confusion matrix using the two human scores (rows=human #1, columns=human #2)."
                             " Note that the matrix is computed on the subset of responses that were "
                             "double-scored.")
        if continuous_human_score:
            markdown_strs.append("Note: Human scores have beeen rounded to the nearest integer.")
        display(Markdown('\n'.join(markdown_strs)))
        df_confmat_h1h2 = DataReader.read_from_file(confmat_h1h2_file, index_col=0)
        display(HTML(df_confmat_h1h2.to_html(float_format=float_format_func, 
                                             formatters=formatter_dict, escape=False)))

    markdown_strs = ['### Disattenuated correlations']
    markdown_strs.append('The next table shows the correlations between human and machine scores, '
                         'the correlations between two human scores, '  
                         'and disattenuated correlations between human and machine scores computed as '
                         'human-machine correlations divided by the square root of human-human '
                         'correlation. '
                         'Note that the human-machine correlation is computed on the full '
                         'dataset (to get a reliable estimate) whereas the human-human '
                         'correlation is computed on the subset of responses that were double-scored.')
    markdown_strs.append("\nThe following values are "
                         "<span class='highlight_color'>highlighted</span>")
    markdown_strs.append(' - `disattenuated_corr` < 0.9')
    display(Markdown('\n'.join(markdown_strs)))
    # display the HTML for the table with the various formatters
    formatter_dis_corr = partial(color_highlighter, low=0.9)
    formatter_dict = {'corr_disattenuated': formatter_dis_corr}
    display(HTML(df_dis_corrs.to_html(index=True,
                                      escape=False,
                                      classes=['sortable'],
                                      float_format=float_format_func,
                                      formatters=formatter_dict)))
else:  
    markdown_strs.append("The configuration file did not specify "
                         "`second_human_score_column` which is necessary to compute "
                         "consistency metrics.")
    display(Markdown('\n'.join(markdown_strs)))
    
    